# Full Text Frequency for BookInXisto search
## Script to generate a frequency table from input xlsx file
### BookInXisto - Getin
### Monique Borges <monique@ua.pt>, Gonçalo Barros, Fillipe Feitosa <fillipefeitosa@ua.pt>


In [24]:
# List of Imports
import pandas as pd
import json
import datetime
from urllib import parse
from urllib.parse import parse_qs, urlparse

In [25]:
# load data
# Only FullText
# fullText = pd.read_excel('fulltext.xlsx')
# Complete Data
fullText = pd.read_excel('Source-limpissimo.xlsx')

In [26]:
fullText

,pagePath,pageTitle,source,users,newUsers,localizar ID]=&,fulltext]=&,[village_id]=&,[type_id]=&,ligação outputs,fulltext_texto,vilalgeid
0,/de/experience-catalog/experience?id=140&Exper...,Integrity constraint violation (#23000),google,1,1,NaN,NaN,NaN,NaN,Restaurante Bem me Quer - Menu DegustaÃ§Ã£o,NaN,NaN
1,/de/experience-catalog/experience?id=144&Exper...,Reise zu Fuß durch die Schieferdörfer (Aldeias...,google,1,1,NaN,NaN,NaN,NaN,Viagem a pÃ© pelas Aldeias do Xisto,NaN,NaN
2,/de/experience-catalog/experience?id=155&Exper...,Restaurante Santo Amaro # 2 Teilnehmer,google,1,1,NaN,NaN,NaN,NaN,Restaurante Santo Amaro - Menu DegustaÃ§Ã£o,NaN,NaN
3,/de/experience-catalog/experience?id=157,Restaurante Varanda do Casal - Menu Degustação...,aldeiasdoxisto.pt,1,1,NaN,NaN,NaN,NaN,Restaurante Varanda do Casal - Menu DegustaÃ§Ã£o,NaN,NaN
4,/de/experience-catalog/experience?id=157&Exper...,Restaurante Varanda do Casal - Menu Degustação...,google,1,1,NaN,NaN,NaN,NaN,Restaurante Varanda do Casal - Menu DegustaÃ§Ã£o,NaN,NaN
5,/de/experience-catalog/experience?id=22,Zu den Wasserfällen # 2 Teilnehmer,google,1,1,NaN,NaN,NaN,NaN,Pelas Cascatas,NaN,NaN
6,/de/experience-catalog/experience?id=22&Experi...,Zu den Wasserfällen # 2 Teilnehmer,google,1,1,NaN,NaN,NaN,NaN,Pelas Cascatas,NaN,NaN
7,/de/experience-catalog/experience?id=235&Exper...,Workshop Keramikfiguren # 2 Teilnehmer,cse.newshub360.de,1,1,NaN,NaN,NaN,NaN,Workshop de Figurado em CerÃ¢mica,NaN,NaN
8,/de/experience-catalog/experience?id=246&Exper...,Restaurante Fiado - Menu Espírito do Lugar # 2...,google,2,1,NaN,NaN,NaN,NaN,Restaurante Fiado - Menu EspÃ­rito do Lugar,NaN,NaN
9,/de/experience-catalog/experience?id=252&Exper...,Restaurante Santo Amaro - Menu Espírito do Lug...,google,1,1,NaN,NaN,NaN,NaN,Restaurante Santo Amaro - Menu EspÃ­rito do Lugar,NaN,NaN


In [27]:
# Routine to calculate a duration, given to date strings. 
def calculateDuration(startDate, endDate):
#     print("-------")
#     print(str(startDate) + ' - ' + str(endDate))
    startDateDelta = datetime.datetime.strptime(startDate, "%Y-%m-%d")
    endDateDelta = datetime.datetime.strptime(endDate, "%Y-%m-%d")
    duration = endDateDelta - startDateDelta
#     print("-------")
    return duration.days

In [28]:
# Split url data
hash_map = {}
# Creating another hash layer to calculate time
listOfParams = []
denayDateParams = ['experiencesearch[date_end]','experiencesearch[date_start]','lodgesearch[date_start]', 'lodgesearch[date_end]', 'LodgeSearch[date_start]','LodgeSearch[date_end]', 'ExperienceSearch[date_start]','ExperienceSearch[date_end]']
for index, row in fullText.iterrows():
    url = parse.urlsplit(row.pagePath)
#     print(url.path)
    params = parse_qs(url.query, keep_blank_values=True)
    listOfParams.append(params)
#     print(params)
    for key in params.keys():
        # Date related keys
        if key not in denayDateParams:
            hash_map[key.lower()] = {}         

In [43]:
# Calculate Order Durations
hash_map['LodgeSearch_duration'] = {}
hash_map['ExperienceSearch_duration'] = {}
hash_map['Index_duration'] = {}
hash_map['id_LodgeSearch'] = {}
hash_map['id_ExperienceSearch'] = {}
hash_map['id_Index'] = {}
hash_map['controlled_VillageId'] = {}
hash_map['controlled_VillageId']['experience'] = {}
hash_map['controlled_VillageId']['lodge'] = {}

# Small Hack to user lower case on DataFrame pagePath
pagePathLower = pd.DataFrame(fullText['pagePath'].str.lower())

for index, row in pagePathLower.iterrows():
    # Getting dataframe.users for the current index
    weightedSum = 1*fullText.iloc[index].newUsers
    expSearch = ['experiencesearch[date_start]', 'experiencesearch[date_end]']
    lodgeSearch = ['lodgesearch[date_start]', 'lodgesearch[date_end]']
    url = parse.urlsplit(row.pagePath)
    params = parse_qs(url.query, keep_blank_values=True)
    
    # Clear empty Data
    if 'id' not in params.keys():
        params['id'] = 'n'
    
    try:    
        # ID Frequency Verification
        if url.path.endswith('experience'):
            # Count Id Frequency for ExperienceSearch
            if params['id'][0] in hash_map['id_ExperienceSearch']:
                hash_map['id_ExperienceSearch'][params['id'][0]] = hash_map['id_ExperienceSearch'][params['id'][0]] + weightedSum
            else:
                hash_map['id_ExperienceSearch'][params['id'][0]] = weightedSum
        if url.path.endswith('lodge'):
            # Count Id Frequency for LodgeSearch
            if params['id'][0] in hash_map['id_LodgeSearch']:
                hash_map['id_LodgeSearch'][params['id'][0]] = hash_map['id_LodgeSearch'][params['id'][0]] +weightedSum
            else:
                hash_map['id_LodgeSearch'][params['id'][0]] = weightedSum
                
        if url.path.endswith('index'):
            if params['id'][0] in hash_map['id_Index']:
                hash_map['id_Index'][params['id'][0]] = hash_map['id_Index'][params['id'][0]] + weightedSum
            else:
                hash_map['id_Index'][params['id'][0]] = weightedSum


        # If Date is related to EXPERIENCE
        if expSearch[0] in params:

            daysExp = calculateDuration(params[expSearch[0]][0], params[expSearch[1]][0])
                
            if daysExp in hash_map['ExperienceSearch_duration']:
                hash_map['ExperienceSearch_duration'][daysExp] = hash_map['ExperienceSearch_duration'][daysExp] + weightedSum
            else:
                hash_map['ExperienceSearch_duration'][daysExp] = weightedSum       

        # If Date is related to LODGE
        elif lodgeSearch[0] in params:
            
            dayLodge = calculateDuration(params[lodgeSearch[0]][0], params[lodgeSearch[1]][0])
            
            if dayLodge in hash_map['LodgeSearch_duration']:
                hash_map['LodgeSearch_duration'][dayLodge] = hash_map['LodgeSearch_duration'][dayLodge] + weightedSum
            else:
                hash_map['LodgeSearch_duration'][dayLodge] = weightedSum
    except:
        print('ERRO: ' + str(index) + ' - ' + str(row))
        
        
        

['1', 'bem']
<class 'list'>
['1', 'bem']
<class 'list'>
['1', 'brandão']
<class 'list'>
['1', 'brandão']
<class 'list'>
['1', 'brandão']
<class 'list'>
['1', 'buke']
<class 'list'>
['1', 'buke']
<class 'list'>
['1', 'buke']
<class 'list'>
['1', 'fiado']
<class 'list'>
['1', 'fiado']
<class 'list'>
['1', 'fiado']
<class 'list'>
['1', 'museu']
<class 'list'>
['1', 'museu']
<class 'list'>
['1', 'museu']
<class 'list'>
['1', 'sabores']
<class 'list'>
['1', 'sabores']
<class 'list'>
['1', 'sabores']
<class 'list'>
['1', 'sabores']
<class 'list'>
['1', 'sabores']
<class 'list'>
['1', 'sabores']
<class 'list'>
['1', 'ti augusta']
<class 'list'>
['1', 'varanda']
<class 'list'>
['1', 'varanda']
<class 'list'>
['1', 'varanda']
<class 'list'>
['1', 'velha']
<class 'list'>
['', '']
<class 'list'>
DEBUG!!!
['', 'jazz']
<class 'list'>
['', 'jazz']
<class 'list'>
['', 'jazz']
<class 'list'>
['', 'jazz']
<class 'list'>
['', 'jazz']
<class 'list'>
['', 'lagarada']
<class 'list'>
['', 'viagem']
<class '

In [42]:
hash_map

{'id': {'140': 28,
  '144': 19,
  '155': 16,
  '157': 30,
  '22': 72,
  '235': 15,
  '246': 9,
  '252': 4,
  '282': 5,
  '31': 30,
  '32': 42,
  '1': 85,
  '11': 14,
  '14': 24,
  '17': 17,
  '19': 40,
  '2': 98,
  '29': 40,
  '35': 87,
  '40': 53,
  '45': 25,
  '47': 66,
  '51': 41,
  '52': 50,
  '53': 21,
  '55': 47,
  '57': 74,
  '6': 42,
  '62': 18,
  '67': 30,
  '78': 9,
  '79': 41,
  '80': 101,
  '87': 13,
  '37': 84,
  '146': 13,
  '147': 16,
  '149': 18,
  '150': 21,
  '151': 14,
  '154': 12,
  '156': 1,
  '206': 27,
  '214': 2,
  '215': 10,
  '241': 22,
  '247': 22,
  '251': 30,
  '254': 12,
  '4': 35,
  '10': 11,
  '12': 16,
  '13': 28,
  '16': 12,
  '18': 13,
  '20': 12,
  '23': 28,
  '24': 29,
  '25': 21,
  '26': 64,
  '27': 46,
  '28': 25,
  '34': 48,
  '36': 22,
  '38': 46,
  '39': 25,
  '43': 29,
  '44': 35,
  '48': 20,
  '49': 40,
  '5': 64,
  '50': 16,
  '58': 70,
  '59': 48,
  '60': 32,
  '61': 57,
  '63': 44,
  '64': 25,
  '66': 48,
  '68': 142,
  '71': 44,
  '72': 9

In [31]:
for param in listOfParams:
    for item in param:
        currentSearch = str(param[item][0]).lower()
        if item not in denayDateParams:
            if(currentSearch in hash_map[item.lower()]):
                hash_map[item.lower()][currentSearch] = hash_map[item.lower()][currentSearch] + 1
            else:
                hash_map[item.lower()][currentSearch] = 1
#         print(param)
#         hash_map[item][] = param[item][0]

In [32]:
hash_map.keys()
aNewHope = {}
aNewHope['id_LodgeSearch'] = hash_map['id_LodgeSearch']
aNewHope['id_ExperienceSearch'] = hash_map['id_ExperienceSearch']
aNewHope['id_Index'] = hash_map['id_Index']
aNewHope['Index_duration'] = hash_map['Index_duration']
aNewHope['LodgeSearch_duration'] = hash_map['LodgeSearch_duration']
aNewHope['ExperienceSearch_duration'] = hash_map['ExperienceSearch_duration']
aNewHope['controlled_VillageId']['experience'] = hash_map['controlled_VillageId']['experience']
aNewHope['controlled_VillageId']['lodge'] = hash_map['controlled_VillageId']['lodge']


KeyError: 'controlled_VillageId'

In [ ]:
# json.dumps(aNewHope, ensure_ascii=False)
hash_map

In [ ]:
aNewHope

In [ ]:
listOfParams